# 0. Set up
## 0.1 If if needed, clone Demeter with git

In [ ]:
!git clone --branch feature/002-rotation https://github.com/antonfrancois/Demeter_metamorphosis.git

and install the package

In [ ]:
import subprocess
import sys

cwd = subprocess.check_output("pwd", text=True).strip()
if not "Demeter" in cwd:
	%cd Demeter_metamorphosis/


print("\nInstalation of demeter from pip")
!pip install -e .
# !pip install -e .

repo_path = os.path.abspath('.')
# repo_path = "/content/Demeter_metamorphosis/"
if repo_path not in sys.path:
	sys.path.insert(0, repo_path)
print(sys.path)


## 0.2 After jupyter restart, run :

In [ ]:


import sys
import os


# Remplace par le chemin relatif où se trouve le package 'demeter'
# Exemple : si ton notebook est dans le dossier root du repo, et demeter est un sous-dossier
import subprocess
cwd = subprocess.check_output("pwd", text=True).strip()
if not "Demeter" in cwd:
	%cd Demeter_metamorphosis/

# repo_path = os.path.abspath('.')
repo_path = "/content/Demeter_metamorphosis/"
if repo_path not in sys.path:
	sys.path.insert(0, repo_path)
print(sys.path)



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 1. code utile

## imports

In [1]:
import torch

import brats_utils as bu
import demeter.utils.torchbox as tb
from demeter.constants import *
import demeter.utils.image_3d_plotter as i3p
import demeter.metamorphosis.rotate as mtrt
import demeter.metamorphosis as mt
import demeter.utils.cost_functions as cf
import demeter.utils.reproducing_kernels as rk

import pandas as pd
import nibabel as nib




/home/turtlefox/anaconda3/envs/demeter_env/lib/python3.12/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


saves_overview.csv


In [15]:
%load_ext autoreload
%autoreload 2
from execute_meta_on_bratsReg2022 import  exe_i_list

brats_list = [
        "BraTSReg_086",
        #"BraTSReg_090","BraTSReg_084",
        # "BraTSReg_046",
        # "BraTSReg_002","BraTSReg_021",
        # "BraTSReg_040","BraTSReg_118","BraTSReg_114","BraTSReg_132",

        # "BraTSReg_101","BraTSReg_073","BraTSReg_025","BraTSReg_022","BraTSReg_068","BraTSReg_120","BraTSReg_031","BraTSReg_088","BraTSReg_006","BraTSReg_003","BraTSReg_024","BraTSReg_035","BraTSReg_076","BraTSReg_012","BraTSReg_123",
    # 'BraTSReg_034',
        # 'BraTSReg_048', 'BraTSReg_055', 'BraTSReg_082', 'BraTSReg_045', 'BraTSReg_089', 'BraTSReg_057',
        # 'BraTSReg_096', 'BraTSReg_083', 'BraTSReg_042', 'BraTSReg_061', 'BraTSReg_074', 'BraTSReg_097', 'BraTSReg_056', 'BraTSReg_033', 'BraTSReg_136', 'BraTSReg_119', 'BraTSReg_108', 'BraTSReg_054', 'BraTSReg_091', 'BraTSReg_100', 'BraTSReg_030', 'BraTSReg_126', 'BraTSReg_133', 'BraTSReg_138', 'BraTSReg_053', 'BraTSReg_110', 'BraTSReg_079',
    # 'BraTSReg_008', 'BraTSReg_131', 'BraTSReg_001', 'BraTSReg_023', 'BraTSReg_064', 'BraTSReg_067', 'BraTSReg_115', 'BraTSReg_029', 'BraTSReg_093', 'BraTSReg_129', 'BraTSReg_005',
    #  'BraTSReg_140',
        #'BraTSReg_036', 'BraTSReg_071'
    ]

device = 'cuda:0'
valid = False  
brats_folder= '2022_valid' if valid else '2022_train'
modality = 'flair'
pb = bu.parse_brats(brats_list=brats_list,brats_folder=brats_folder,modality=modality)
save_folder = None
scale_img = 1

def foo(img1,img2,seg_1,seg_2):
    return 0
i_list = torch.arange(len(pb.brats_list))
print(type(i_list))
exe_i_list(foo, pb, i_list,valid, modality, 
           scale_img = scale_img,
           save_path=save_folder,
           try_to_load_mask=False,
           device = device
           )

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
<class 'torch.Tensor'>

Results will be saved at :/home/turtlefox/Documents/11_metamorphoses/Demeter_metamorphosis/examples/results/bratsReg2022_20250527_1629/optim_overview.csv

i list tensor([0]) <class 'torch.Tensor'>

==== openning 0:BraTSReg_086  1/1======
scale_img :  1


ic| execute_meta_on_bratsReg2022.py:550 in exe_i_list()
    (tensor([[ 76., 119., 121.],
            [ 81., 140., 111.],
            [ 82., 127., 127.],
            [ 77., 135., 121.],
            [ 79., 103.,  66.],
            [ 65., 149., 122.],
            [ 60., 115., 136.],
            [ 60., 103., 148.],
            [ 60.,  99., 119.],
            [ 58.,  98., 155.]]),
     tensor([[ 75., 119., 125.],
            [ 80., 139., 114.],
            [ 82., 127., 129.],
            [ 77., 135., 123.],
            [ 77., 101.,  66.],
            [ 65., 148., 120.],
            [ 59., 115., 138.],
            [ 59., 104., 148.],
            [ 59.,  99., 119.],
            [ 59.,  97., 155.]]))
ic| execute_meta_on_bratsReg2022.py:556 in exe_i_list()- 'mr computed'
ic| execute_meta_on_bratsReg2022.py:557 in exe_i_list()- 28
ic| execute_meta_on_bratsReg2022.py:566 in exe_i_list()
    'Land_dist will be computed'


image shape : torch.Size([1, 1, 240, 240, 155])


AttributeError: 'int' object has no attribute 'compute_landmark_dist'

In [ ]:

class ExeMeta_rigid():

    def __init__(self,n_steps = None,
                 rho = None,
                 lamb = None,
                 kernelOperator =  None,
                 sharp = None,
                 n_iter=None,
                 dx_convention = None,
                 ):
        self.n_steps = 10   if n_steps is None else n_steps
        self.lamb = 1e-7    if lamb is None else lamb
        self.rho = 10  if rho is None else rho
        self.sharp = True   if sharp is None else sharp
        self.n_iter = 1000 if n_iter is None else n_iter
        if kernelOperator is None:
            sigma = [(3,3),(7,7)]
            self.kernelOperator = rk.Multi_scale_GaussianRKHS(
                sigma,
                normalized=False,
            )
        else:
            self.kernelOperator = kernelOperator
        self.dx_convention = "pixel" if dx_convention is None else dx_convention

    def __call__(self,img_1,img_2,seg_1,seg_2):
        n_iter = self.n_iter if at_serv else 3
        mr = mt.metamorphosis(img_2,img_1,0,
                              self.rho,self.lamb,self.n_steps,
                              kernelOperator=self.kernelOperator,
                              n_iter = n_iter,grad_coef=.1,sharp=self.sharp,
                             dx_convention=self.dx_convention
                              )
        return mr